In [1]:
import turtle
import random
import numpy as np
from keras import Sequential
from collections import deque
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.optimizers import Adam
import pickle
import tensorflow as tf
import time
grid = np.linspace(-240, 240, 49)

In [2]:
grid

array([-240., -230., -220., -210., -200., -190., -180., -170., -160.,
       -150., -140., -130., -120., -110., -100.,  -90.,  -80.,  -70.,
        -60.,  -50.,  -40.,  -30.,  -20.,  -10.,    0.,   10.,   20.,
         30.,   40.,   50.,   60.,   70.,   80.,   90.,  100.,  110.,
        120.,  130.,  140.,  150.,  160.,  170.,  180.,  190.,  200.,
        210.,  220.,  230.,  240.])

In [3]:
class TurtleGame():
    
    def __init__(self):
        
        self.done = False
        self.reward = 0
        self.count = 0
        self.gameover = False
        self.level = 1
        
        #background
        
        self.win = turtle.Screen()
        self.win.title('Bad')
        self.win.bgcolor('Black')
        self.win.tracer(0)
        self.win.setup(width=600, height = 600)
        
        #animal
        
        self.animal = turtle.Turtle()
        self.animal.shape('turtle')
        self.animal.direction = 'stop'
        self.animal.color('yellow')
        self.animal.shapesize(stretch_wid=1, stretch_len=1)
        self.animal.penup()
        self.animal.goto(random.choice(grid),random.choice(grid))
        
        
        #food
        self.foods = []
        
        for _ in range(3):
            self.food = turtle.Turtle()
            self.food.shape('circle')
            self.food.penup()
            self.food.color('orange')
            
            self.food.goto(random.choice(grid),random.choice(grid))
            
            self.foods.append(self.food)
        
        #keyboard
        
        self.win.listen()
        self.win.onkeypress(self.animal_right, 'Right')   
        self.win.onkeypress(self.animal_left, 'Left')
        self.win.onkeypress(self.animal_up, 'Up')   
        self.win.onkeypress(self.animal_down, 'Down')
        
        
        #distance
        
        self.distance = np.abs(self.animal.distance(self.food.xcor(), self.food.ycor()))
        
        #lives and score
        
        self.lives = 1
        self.score = 0
        
        #showing lives and score
        self.pen = turtle.Turtle()
        self.pen.speed(0)
        self.pen.color('blue')
        self.pen.penup()
        self.pen.goto(0,230)
        self.pen.pendown()
        self.pen.write('Score: {}  Lives:'.format(self.score, self.lives), align = 'center', font=('Courier', 20))
        self.pen.hideturtle()
        

    
        # Create a turtle object for drawing the maze
        self.penm = turtle.Turtle()
        self.penm.speed(0)
        self.penm.color("blue")
        self.penm.penup()
        self.penm.hideturtle()

        
        #enemies
        self.enemies = []
        for x in range(25):
            self.enemy = turtle.Turtle()
            self.enemy.penup()
            self.enemy.color('red')
            self.enemy.shape('circle')
            self.enemy.shapesize(stretch_wid=0.5, stretch_len=0.5)
            self.enemy.speed = 10
            self.enemy.goto(random.choice(grid),random.choice(grid))
            self.enemies.append(self.enemy)
            self.enemy.frame_count = 0
            
    #pacman movement
    def movement(self):
        
        if self.animal.direction == 'right':
            
            x = self.animal.xcor() 

            if self.animal.heading() != 0.0:
                self.animal.setheading(0.0)

            if x < 240:

                self.animal.setx(x+10)
                
        if self.animal.direction == 'left':
            
            x = self.animal.xcor()   

            if self.animal.heading() != 180.0:
                self.animal.setheading(180.0)

            if x > -240:
                self.animal.setx(x-10)  

        if self.animal.direction == 'up':
            
            y = self.animal.ycor()

            if self.animal.heading() != 90.0:
                self.animal.setheading(90.0)

            if y < 240:
                self.animal.sety(y+10)
            
        if self.animal.direction == 'down':
            
            y = self.animal.ycor()

            if self.animal.heading() != 270.0:
                self.animal.setheading(270.0)

            if y > -240:
                
                self.animal.sety(y-10)
    def move_enemies(self):
        
        for enemy in self.enemies:
            
            if enemy.frame_count == 0:
                enemy.direction = random.choice(['up', 'down', 'left', 'right'])
                enemy.frame_count = random.randint(100, 130)
            else:
                enemy.frame_count -= 1

            if enemy.direction == 'up':
                if enemy.ycor() + enemy.speed > 240:
                    enemy.direction = 'down'
                else:
                    enemy.sety(enemy.ycor() + enemy.speed)
            elif enemy.direction == 'down':
                if enemy.ycor() - enemy.speed < -240:
                    enemy.direction = 'up'
                else:
                    enemy.sety(enemy.ycor() - enemy.speed)
            elif enemy.direction == 'left':
                if enemy.xcor() - enemy.speed < -240:
                    enemy.direction = 'right'
                else:
                    enemy.setx(enemy.xcor() - enemy.speed)
            elif enemy.direction == 'right':
                if enemy.xcor() + enemy.speed > 240:
                    enemy.direction = 'left'
                else:
                    enemy.setx(enemy.xcor() + enemy.speed)
        
    def animal_right(self):
        
        self.animal.direction = 'right'
        
    def animal_left(self):
        
        self.animal.direction = 'left'

    def animal_down(self):
        
        self.animal.direction = 'down'
            
    def animal_up(self):
        
        self.animal.direction = 'up'
    
    #window update
    
    def run_frame(self):

        self.win.update()

        for food in self.foods:
            if self.animal.distance(food) < 5:
                self.foods.remove(food)
                food.hideturtle()
                self.count += 1
                self.score += 1
                self.pen.clear()
                self.pen.write('Score: {}  Lives: {}'.format(self.score, self.lives), align = 'center', font=('Courier', 20))
        
            if self.count == 6:
                self.done = True
                self.reset()
                self.count = 0
                self.score = 0
                self.lives = 1
                self.pen.clear()
                self.pen.write('Score: {}  Lives: {}'.format(self.score, self.lives), align = 'center', font=('Courier', 20))
        self.enemy_movement()
 
    #AI movement
    
    def reset(self):
        
        for food in self.foods:

            food.hideturtle()
            
              
        #self.animal.goto(random.choice(grid),random.choice(grid))
        self.foods = []
        
        for _ in range(6):
            
            self.food = turtle.Turtle()
            self.food.shape('circle')
            self.food.penup()
            self.food.color('orange')
            
            self.food.goto(random.choice(grid),random.choice(grid))
            
            self.foods.append(self.food)
            
        return [self.animal.xcor(), self.animal.ycor(), self.food.xcor(), self.food.ycor(), 
                self.food.xcor() - self.animal.xcor(), self.food.ycor() - self.animal.ycor()]
    
    def step(self, action):
        
        self.done = 0
        self.reward = 0
        
        dist_in = np.abs(self.animal.distance(self.food.xcor(), self.food.ycor()))
        
        if action == 0:
            self.animal_left()
            self.reward -= 3
            if self.animal.xcor() == -240:
                self.reward -= 10
        if action == 1:
            self.animal_right()
            self.reward -= 3
            if self.animal.xcor() == +240:
                self.reward -= 10
        if action == 2:
            self.animal_down()
            self.reward -= 3
            if self.animal.ycor() == -240:
                self.reward -= 10
        if action == 3:
            self.animal_up()
            self.reward -= 3
            if self.animal.ycor() == +240:
                self.reward -= 10
        self.distance = np.abs(self.animal.distance(self.food.xcor(), self.food.ycor()))
        dist_fin = np.abs(self.animal.distance(self.food.xcor(), self.food.ycor()))
        
        if dist_fin == 0:
            self.reward += 400

        if (dist_fin - dist_in > 0):
            self.reward -= 10
            
        elif (dist_fin - dist_in < 0):
            self.reward += 5
       
        self.run_frame()

        state = [self.animal.xcor(), self.animal.ycor(), self.food.xcor(), self.food.ycor(), 
                 self.food.xcor() - self.animal.xcor(), self.food.ycor() - self.animal.ycor()]
        return state, self.reward, self.done



In [4]:
env = TurtleGame()
while True:
    time.sleep(0.1)
    env.win.update()
    

    for food in env.foods:
        
        if env.animal.distance(food) < 10:
            #env.foods.remove(food)
            food.goto(random.choice(grid),random.choice(grid))
            #env.foods.append(food)
            env.count += 1
            env.score += 1
            env.pen.clear()
            env.pen.write('Score: {}  Lives: {} '.format(env.score, env.lives), align = 'center', font=('Courier', 20))

#        if env.count == 6:
#            env.done = True
#            env.reset()
#            env.level += 1
#            env.count = 0
#            env.score = 0
#           env.lives = 1
#            env.pen.clear()
#            env.pen.write('Score: {}  Lives: {} '.format(env.score, env.lives), align = 'center', font=('Courier', 20))
    
    for enemy in env.enemies:
        
        if env.animal.distance(enemy) < 5 and env.lives != 0:
            
            env.lives -= 1
            env.pen.clear()
            env.pen.write('Score: {}  Lives: {}'.format(env.score, env.lives), align = 'center', font=('Courier', 20))
            time.sleep(1)
            env.animal.goto(random.choice(grid),random.choice(grid))

    if env.lives == 0:
        
        env.done = True
        env.reset()
        env.level = 1
        env.count = 0
        env.score = 0
        env.lives = 1
        game_over = turtle.Turtle()
        game_over.speed(0)
        game_over.color('yellow')
        game_over.penup()
        game_over.hideturtle()
        game_over.goto(0, 0)
        game_over.write('Game Over', align='center', font=('Courier', 40, 'bold'))
        time.sleep(2)
        game_over.clear()
        env.pen.clear()
        env.pen.write('Score: {}  Lives: {}'.format(env.score, env.lives), align = 'center', font=('Courier', 20))

        
        
    env.movement()     
    env.move_enemies()
    

    

TclError: invalid command name ".!canvas"

In [ ]:
class DQN:

    def __init__(self, action_space, state_space):

        self.action_space = action_space
        self.state_space = state_space
        self.epsilon = 1
        self.gamma = .95
        self.batch_size = 64
        self.epsilon_min = .01
        self.epsilon_decay = .995
        self.learning_rate = 0.001
        self.memory = deque(maxlen=100000)
        self.model = self.build_model()

    def build_model(self):

        model = Sequential()
        model.add(Dense(64, input_shape=(self.state_space,), activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):

        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_space)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def act_trained(self, state):
        if np.random.rand() < self.epsilon:
            # Random action
            return random.randrange(self.action_space)
        else:
            # Greedy action
            return np.argmax(self.model.predict(state)[0])


    def replay(self):

        if len(self.memory) < self.batch_size:

            return

        minibatch = random.sample(self.memory, self.batch_size)
        states = np.array([i[0] for i in minibatch])
        actions = np.array([i[1] for i in minibatch])
        rewards = np.array([i[2] for i in minibatch])
        next_states = np.array([i[3] for i in minibatch])
        dones = np.array([i[4] for i in minibatch])

        states = np.squeeze(states)
        next_states = np.squeeze(next_states)

        targets = rewards + self.gamma*(np.amax(self.model.predict_on_batch(next_states), axis=1))*(1-dones)
        targets_full = self.model.predict_on_batch(states)

        ind = np.array([i for i in range(self.batch_size)])
        targets_full[[ind], [actions]] = targets

        self.model.fit(states, targets_full, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def save_agent(self, file_path):
       
        self.model.save(file_path + "_model.h5")
    
        with open(file_path + "_epsilon.pkl", "wb") as f:
            pickle.dump(self.epsilon, f)
    
    @classmethod
    
    def load_agent(cls, action_space, state_space, file_path):
        
        loaded_model = tf.keras.models.load_model(file_path + "_model.h5")

        with open(file_path + "_epsilon.pkl", "rb") as f:
            
            loaded_epsilon = pickle.load(f)
            
        new_agent = cls(action_space=action_space, state_space=state_space)

        new_agent.model = loaded_model
        new_agent.epsilon = loaded_epsilon

        return new_agent

def train_dqn(episode):

    loss = []

    action_space = 4
    state_space = 6
    max_steps = 1000

    agent = DQN(action_space, state_space)
    for e in range(episode):
        state = env.reset()
        state = np.reshape(state, (1, state_space))
        score = 0
        for i in range(max_steps):
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            score += reward
            next_state = np.reshape(next_state, (1, state_space))
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            agent.replay()
            if done:
                print("episode: {}/{}, score: {}".format(e, episode, score))
                break
        loss.append(score)
    return agent, loss


def train_dqn_again(episode, agent):

    loss = []

    action_space = 4
    state_space = 6
    max_steps = 1000
    
    for e in range(episode):
        state = env.reset()
        state = np.reshape(state, (1, state_space))
        score = 0
        for i in range(max_steps):
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            score += reward
            next_state = np.reshape(next_state, (1, state_space))
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            agent.replay()
            if done:
                print("episode: {}/{}, score: {}".format(e, episode, score))
                break
        loss.append(score)
    return agent, loss





def test_dqn(episode, agent):
    action_space = 4
    state_space = 6
    max_steps = 100

    for e in range(episode):
        state = env.reset()
        state = np.reshape(state, (1, state_space))
        score = 0
        for i in range(max_steps):
            action = agent.act_trained(state)
            next_state, reward, done = env.step(action)
            score += reward
            next_state = np.reshape(next_state, (1, state_space))
            state = next_state
            if done:
                print("episode: {}/{}, score: {}".format(e, episode, score))
                break

    return 